# Testing the `channel` function given in the pdf

In [1]:
import numpy as np
import math

In [2]:
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3) 
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput)), erasedIndex

In [3]:
channelInput = np.array([1,-1,1,1,1,-1])
channelOutput = channel(channelInput)

print(channelOutput)

(array([1.98366069, 0.7608189 , 0.5196792 , 3.97914553, 2.59692904,
       0.30218161]), 1)


# Command to get the real channel output

In [4]:
python3 client.py --input_file=in.txt --output_file=out.txt --srv_hostname iscsrv72.epfl.ch --srv_port 80

SyntaxError: invalid syntax (<ipython-input-4-2c4899140df9>, line 1)

In [5]:
np.savetxt("in.txt", channelInput)

In [6]:
channelOutput = np.loadtxt("out.txt")
print(channelOutput)

[ 8.86527571  2.56791895 -5.904792   -2.13201648  8.97192555 -0.96314209]


In [7]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def stringToBits(string):
    bits = tobits(string)
    bits = [bits[i] for i in range(len(bits)) if i%8!=0]
    bits = np.array(bits, dtype='int64')    
    
    return bits

def channelOutputToString(channelOutput):
    bits = channelOutput.astype('int64').tolist()

    n = 7
    i = 0
    while i < len(bits):
        bits.insert(i, 0)
        i += (n+1)

    return frombits(bits)

In [8]:
def getRandom1ByteUtf8():
    return [0] + [random.randint(0,1) for i in range(7)]

def getRandomString(length=80):
    packs = [random.randint(0,1) for i in range(length)]
    bits = []
    for i in range(length):
        bits += getRandom1ByteUtf8()

    return frombits(bits)

def strDistance(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            count += 1
    return count

# Encoding

In [9]:
codewords = []

def inverseSign(word):
    return [-i for i in word]

def genCodebook(initial=[[1]], k=5):
    if k == 0:
        return initial
    else:
        new = [word + inverseSign(word) for word in initial] + [word + word for word in initial]
        return genCodebook(new, k-1)

def pruneFirstInCodebook(codebook):
    return [codeword[1:] for codeword in codebook]

In [10]:
blockCodebook = genCodebook(initial=[[1]], k=9)


pairCodebook = [[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1]]
tupleToPairCodebookDict = {(1, 1): pairCodebook[0], (1, -1): pairCodebook[1], (-1, 1): pairCodebook[2], (-1, -1): pairCodebook[3]}

In [11]:
def arrayAsNumber(l):
    res = 0
    for i in range(len(l)):
        res += l[i] * (2**i)

    return int(res)

def expandBlock(block):
    return blockCodebook[arrayAsNumber(block)]

# The channel only accepts -1 and 1
def transformPair(subarray):
    return tupleToPairCodebookDict[tuple(subarray)]

# We assume that the array is of even length
def expandPairs(array):
    n = array.shape[0]
    newArray = np.array([], dtype='int64')
    for i in range(n//2):
        newArray = np.append(newArray, transformPair(array[2*i:2*(i+1)]))

    return newArray

def encode(x, K):
    n = x.shape[0]
    blocksCount = n // K

    print(f"input block no: {n/K}")
    nx = np.array([], dtype='int64')
    for i in range(blocksCount):
        nx = np.append(nx, expandBlock(x[i*K:(i+1)*K]))

    # Add size of padding to the end of the block, by definition less than K
    if n % K != 0:
        paddingSize = (blocksCount+1)*K - n
        print(f"input paddingSize: {paddingSize}")

        zeroPadding = np.zeros((paddingSize))
        paddedEnd = np.concatenate((nx[blocksCount*K:n], zeroPadding))
        nx = np.append(nx, expandBlock(paddedEnd))

        # nx = np.append(nx, blockCodebook[paddingSize])
    else:
        print(f"input paddingSize: {0}")
        # nx = np.append(nx, blockCodebook[0])

    nx = expandPairs(nx)

    return nx
    


In [12]:
def jmap(array):
    p1 = p2 = p3 = 0
    for i in range(len(array)//3):
        p1 += abs(array[3*i])**2
        p2 += abs(array[3*i+1])**2
        p3 += abs(array[3*i+2])**2
    if p1 < p2 and p1 < p3:
        return 0, [p1, p2, p3]
    if p2 < p1 and p2 < p3:
        return 1, [p1, p2, p3]
    return 2, [p1, p2, p3]

# We assume that the array is of length divisible by 3
def cutJ(array, j, K):
    n = array.shape[0]
    index = [i%3!=j for i in range(n)]

    return array[index]

def transformBlock(block, K, debug, returnIndex=False):    
    if debug:
        print("Subarray:")
        print(subarray)

    # Maximum inner product decoding

    chosenIndex = -1
    maxProd = -1
    for i in range(len(blockCodebook)):
        prod = np.inner(block, np.array(blockCodebook[i])) 
        if prod > maxProd:
            chosenIndex = i
            maxProd = prod
    
    # Minimum distance decoding

    # chosenIndex = -1
    # minDist = sys.maxsize
    # for i in range(len(blockCodebook)):
    #     dist = np.linalg.norm(block-np.array(blockCodebook[i]), ord=1) 
    #     if dist < minDist:
    #         chosenIndex = i
    #         minDist = dist

    if returnIndex:
        return chosenIndex
    else:
        litteralBinary = bin(chosenIndex)[2:]
        # originalCodeword = list(reversed([int(x) for x in litteralBinary])) + [0 for i in range(K - len(litteralBinary))]
        originalCodeword = list(reversed([int(x) for x in litteralBinary])) + [0 for i in range(K - len(litteralBinary))]
        return originalCodeword

def deletePadding(array, K, debug):
    if 2**K % K == 0: 
        expectedN = 80 * (2**K // K)
    else: 
        expectedN = 80 * (2**K // K + 1)

    n = array.shape[0]
    paddingSize = n - expectedN

    print(f"n: {n}, expected n: {expectedN}")
    # paddingSize = transformBlock(paddingRegion, K, debug, returnIndex=True)

    print(f"output paddingSize: {paddingSize}")

    return array[0:n-paddingSize]

def reduceSignal(array, j, K, verboseDebug):
    n = array.shape[0]
    newArray = np.array([], dtype='int64')

    blockSize = (2**K)

    print(f"output block no: {n//blockSize}")

    for i in range(n//blockSize):
        newArray = np.append(newArray, transformBlock(array[blockSize*i:blockSize*(i+1)], K, verboseDebug))

    newArray = deletePadding(newArray, K, verboseDebug)

    print(f"output block no adjusted: {newArray.shape[0]/K}")

    return newArray 

# array is an np.array with values -1 and 1.
def decode(array, K=10, debug=False, verbose=False):
    n = array.shape[0]
    j, stats = jmap(array)
    if debug:
        print(f"j: {j}")
    newArray = cutJ(array, j, K)
    if debug:
        print(f"after cutJ: {newArray}, size: {len(newArray)}")

    newArray = reduceSignal(newArray, j, K, debug and verbose)

    return newArray, j, stats

In [13]:
import random

avg_dist = 0
trials = 10
length = 80
K = 9

for i in range(trials):
    inputString = getRandomString(length)
    x = stringToBits(inputString)

    y = encode(x, K)
    print(f"y goodness: {np.in1d([-1, 1], y).all()}")
    print(f"n: {y.shape[0]}")
    y, j = channel(y)
    y, jj, stats = decode(y, K, debug=True)

    print(f"n': {y.shape[0]}")

    # delta = y-x

    # if np.any(delta):
        # diff = np.linalg.norm(delta/2, ord=1)
        # print(f"ERROR: n: {n}, diff: {diff}, j: {j}, infered: {jj}, \n (y-x)/2: {(delta/2)}, \n x: {x}, \n y: {y}")
        # avg_diff += diff 
    # else: 
        # print(f"CORRECT: {np.linalg.norm(delta/2, ord=1)/n}")

    outputString = channelOutputToString(y)

    # print(f"encoded size: {y.shape[0]}, input: {inputString}, output: {outputString}")

    if j != jj:
        errs+=1
        print("ERROR IN J ESTIMATION")
        print(f"j: {j}, infered: {jj}")
        print(f"stats: {stats}")
    # elif not np.array_equal(x, y):
    #     errs+=1
    #     print(f"ERROR: x: {x}, y: {y}")

    dist = strDistance(inputString, outputString)

    print(f"dist: {dist}")
    avg_dist += dist

print(f"Average distance: {avg_dist/trials}")

input block no: 62.22222222222222
input paddingSize: 7
y goodness: True
n: 48384
j: 2
after cutJ: [ 2.14768531  3.21017491 -4.12899105 ...  1.57283335  7.79834225
 -5.30287398], size: 32256
output block no: 63
n: 567, expected n: 4560
output paddingSize: -3993
output block no adjusted: 63.0
n': 567
dist: 1
input block no: 62.22222222222222
input paddingSize: 7
y goodness: True
n: 48384
j: 0
after cutJ: [ 3.26192    -2.29108529 -2.97326991 ...  5.85122652 -2.99484351
  1.9759178 ], size: 32256
output block no: 63
n: 567, expected n: 4560
output paddingSize: -3993
output block no adjusted: 63.0
n': 567
dist: 71
input block no: 62.22222222222222
input paddingSize: 7
y goodness: True
n: 48384
j: 0
after cutJ: [ 3.77027949  9.05744225 -0.8510631  ... -7.82383147  3.01145275
 -0.43748164], size: 32256
output block no: 63
n: 567, expected n: 4560
output paddingSize: -3993
output block no adjusted: 63.0
n': 567
dist: 63
input block no: 62.22222222222222
input paddingSize: 7
y goodness: True
n: